In [1]:
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import torch

import pymorphy2
import re
import spacy

from string import punctuation

data_path = "data/"
COLUMN = "description"

import os
print(os.cpu_count())

64


In [651]:
from clickhouse_driver import Client

def clickRequest(query: str):
    client = Client(
        host = "144.126.249.217",
        port = 9000,
        user = 'smartsales',
        password = 'lMZybloPciKL0pxS1iEjc7rK8kK79hPH',
        database = 'smartsales',
        connect_timeout=180,
        sync_request_timeout=90
    )

    data = client.execute(query)
    
    return data

In [656]:
query = f"""select
        id,
        min(created_ts) as date_arrival,
        anyLast(category_id) as category_id
    from wb_products
    group by id
    HAVING date_arrival >= '2023-08-10' AND date_arrival <= '2023-09-10'
    SETTINGS max_bytes_before_external_group_by = 100000000
"""

clms = ["id", "date_arrival", "category_id"]

df = pd.DataFrame(clickRequest(query), columns=clms)

df.to_parquet("data/products_arrival_08100910.parquet.gzip", index=False, compression="gzip")

In [668]:
query = f"""select
        id,
        max(created_ts) as date_update,
        anyLast(title) as title,
        anyLast(description) as description,
        argMax(price, created_ts) as price,
        argMax(discount_price, created_ts) as discount_price,
        anyLast(supplier_id) as supplier_id,
        argMax(review_rating, created_ts) as review_rating,
        argMax(review_number, created_ts) as review_number,
        argMax(stock_qty, created_ts) as stock_qty,
        anyLast(sale_qty) as sale_qty,
        anyLast(revenue) as revenue,
        anyLast(category_id) as category_id
    from wb_products
    where toDate(created_ts) >= '2023-09-02' AND toDate(created_ts) <= '2023-09-08'
    group by id, toDate(created_ts)
    SETTINGS max_bytes_before_external_group_by = 100000000
"""

clms = ["id", "date_update", "title", "description", "price", "discount_price", "supplier_id", 
    "review_rating", "review_number", "stock_qty", "sale_qty", "revenue", "category_id"]

df = pd.DataFrame(clickRequest(query), columns=clms)

df.to_parquet("data/downloads/products09020908.parquet.gzip", index=False, compression="gzip")

In [366]:
print(1)

1


In [3]:
df = pd.read_parquet(data_path+"wb_products.parquet.gzip")
print(df.shape, df.description.isna().sum())
df = df.dropna(subset=['description']).reset_index(drop=True)
print(df.shape, df.description.isna().sum())

(1831218, 12) 24857
(1806361, 12) 0


In [35]:
# join data done
df_done = pd.concat([
    pd.read_parquet(data_path+"keywords/df_keywords.parquet.gzip"),
    pd.read_parquet(data_path+"keywords/_df_keywords.parquet.gzip")
]).reset_index(drop=True)
print(df_done.shape)

df_done.to_parquet(data_path+"keywords/df_keywords.parquet.gzip", index=False, compression="gzip")

(1811222, 13)


In [36]:
# already processed
df_done = pd.read_parquet(data_path+"keywords/df_keywords.parquet.gzip")
df = df[~df.id.isin(df_done.id.values)].reset_index(drop=True)
print(df.shape)

(0, 12)


In [23]:
del df_done

In [32]:
# clean 
df[COLUMN] = df[COLUMN].apply(lambda v: v.replace("\\n", "").replace("\n", "").replace("\\", "").replace('"', '').replace("-", ""))

In [25]:
def contains_number(input_string):
    return bool(re.search(r'\d', input_string))

def contains_latin_punct_characters(input_string):
    return any(ord(char) < 128 for char in input_string)

def extract_keywords(text, nlp):

    doc = nlp(text)
    keywords = []

    for token in doc:
        if token.text in nlp.Defaults.stop_words:
            continue
        elif token.text in punctuation:
            continue
        elif len(token.text) < 3:
            continue
        elif contains_number(token.text) == True:
            continue
        elif contains_latin_punct_characters(token.text) == True:
            continue
        elif token.pos_ in ["PROPN", 'X']:
            continue
        elif token.pos_ in ["NOUN", "ADJ"] or token.ent_type_:
            keywords.append(token.text.lower())

    return keywords

nlp = spacy.load('ru_core_news_md')
morph = pymorphy2.MorphAnalyzer()

df_keywords = pd.DataFrame(dtype="object")

In [33]:
batch_size = 500
total_batches = (len(df) + batch_size - 1) // batch_size

for i in tqdm(range(total_batches), desc="Processing Batches"):
    dfc = df[i*batch_size:(i+1)*batch_size].reset_index(drop=True)
    dfc["keywords"] = dfc[COLUMN].apply(extract_keywords, nlp=nlp)
    dfc.keywords = dfc.keywords.apply(lambda v: [morph.parse(w)[0].normal_form for w in v])
    dfc.keywords = dfc.keywords.apply(lambda v: " ".join(v))
    df_keywords = pd.concat([df_keywords, dfc]).reset_index(drop=True)
    if i % 10 == 0:
        df_keywords.to_parquet(
            data_path+f"keywords/{i}_df_keywords.parquet.gzip", compression="gzip", index=False
        )
df_keywords.to_parquet(
    data_path+f"keywords/_df_keywords.parquet.gzip", compression="gzip", index=False
)

Processing Batches: 100%|██████████| 7/7 [05:06<00:00, 43.74s/it]


In [18]:
print(1)

1
